In [1]:
%%capture
!git clone https://github.com/ynop/py-ctc-decode
!pip3 install transformers datasets torchaudio
!pip3 install https://github.com/kpu/kenlm/archive/master.zip

In [ ]:
%cd py-ctc-decode

In [ ]:
import torch
import torchaudio
from datasets import load_dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Wav2Vec2CTCTokenizer

test_dataset = load_dataset("common_voice", "id", split="test[:2%]")

processor = Wav2Vec2Processor.from_pretrained("indonesian-nlp/wav2vec2-large-xlsr-indonesian")
tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("indonesian-nlp/wav2vec2-large-xlsr-indonesian")
model = Wav2Vec2ForCTC.from_pretrained("indonesian-nlp/wav2vec2-large-xlsr-indonesian")

# Preprocessing the datasets.
# We need to read the aduio files as arrays
def speech_file_to_array_fn(batch):
  speech_array, sampling_rate = torchaudio.load(batch["path"])
  resampler = torchaudio.transforms.Resample(sampling_rate, 16_000)
  batch["speech"] = resampler(speech_array).squeeze().numpy()
  return batch

test_dataset = test_dataset.map(speech_file_to_array_fn)
inputs = processor(test_dataset[:10]["speech"], sampling_rate=16_000, return_tensors="pt", padding=True)

with torch.no_grad():
  logits = model(inputs.input_values, attention_mask=inputs.attention_mask).logits

predicted_ids = torch.argmax(logits, dim=-1)

In [50]:
# use your own kenlm model
%%capture
!wget http://hetzner.depia.wiki/idwiki.arpa.gz
!gunzip idwiki.arpa.gz

In [51]:
vocab_dict = tokenizer.get_vocab()
sort_vocab = sorted((value, key) for (key,value) in vocab_dict.items())
vocab = [x[1].replace("|", " ") if x[1] not in tokenizer.all_special_tokens else "" for x in sort_vocab]

In [ ]:
import ctcdecode
import numpy as np

vocabulary = vocab
alpha = 2.5 # LM Weight
beta = 0.0 # LM Usage Reward
word_lm_scorer = ctcdecode.WordKenLMScorer('idwiki.arpa', alpha, beta) # use your own kenlm model
decoder = ctcdecode.BeamSearchDecoder(
    vocabulary,
    num_workers=2,
    beam_width=128,
    scorers=[word_lm_scorer],
    cutoff_prob=np.log(0.000001),
    cutoff_top_n=40
)
text = decoder.decode_batch(logits.numpy())

In [62]:
def lm_postprocess(text):
  # my kenlm is borked
  return ' '.join([x if len(x) > 1 else "" for x in text.split()]).strip()

In [ ]:
text = [lm_postprocess(x) for x in text]
print("LM Prediction: ", text)
print("Prediction:", processor.batch_decode(predicted_ids))
print("Reference: ", test_dataset[:10]["sentence"])